---
jupyter: python3
---

In [ ]:
import random
from collections import defaultdict

import pandas as pd
import swifter  # noqa: F401
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from tqdm.notebook import tqdm

In [ ]:
MIN_COMMENTS_PER_SUBSET = 50
N_OF_SPLITS_PER_SUBSET = 50

In [ ]:
df_reddit = pd.read_parquet("../data/processed/reddit_posts.parquet")
df_reddit["created"] = pd.to_datetime(df_reddit["created"], format="%Y-%m-%d %H:%M:%S")
df_reddit["edited"] = pd.to_datetime(df_reddit["edited"], format="%Y-%m-%d %H:%M:%S")

df_reddit.head(1)

In [ ]:
df_coins = pd.read_json("../data/raw/coins.json")
df_coins["start_date"] = pd.to_datetime(df_coins["start_date"], format="ISO8601")
df_coins["end_date"] = pd.to_datetime(df_coins["end_date"], format="ISO8601")
df_coins = df_coins.set_index("name")

df_coins.head(1)

In [ ]:
def get_number_of_comments_per_coin(name: str, cutoff_date: str) -> int:
    """
    Get the number of comments for a given coin.

    Args:
        name (str): The name of the coin.
        cutoff_date (str): The cutoff date for the comments.

    Returns:
        int: The number of comments for the given coin.

    """
    cutoff_date = pd.to_datetime(cutoff_date, format="%Y-%m-%d")
    return len(df_reddit[(df_reddit["search_query"] == name) & (df_reddit["created"] <= cutoff_date)])


def get_first_comment_date(name: str) -> str:
    """
    Get the first comment date for a given coin.

    Args:
        name (str): The name of the coin.

    Returns:
        str: The first comment date for the given coin.

    """
    return df_reddit[df_reddit["search_query"] == name]["created"].min()


def get_last_comment_date(name: str) -> str:
    """
    Get the last comment date for a given coin.

    Args:
        name (str): The name of the coin.

    Returns:
        str: The last comment date for the given coin.

    """
    mod_df_reddit = df_reddit.copy()
    mod_df_reddit["edited"] = mod_df_reddit["edited"].fillna(mod_df_reddit["created"])
    return mod_df_reddit[mod_df_reddit["search_query"] == name]["edited"].max()


def get_coin_info(name: str) -> pd.Series:
    """
    Get coin information for a given coin name.

    Args:
        name (str): The name of the coin.

    Returns:
        pd.Series: The coin information.

    """
    return df_coins.loc[name]


def get_coin_info_row(row: pd.Series) -> pd.Series:
    """
    Get coin information for a given row.

    Args:
        row (pd.Series): A row from the dataframe containing a search_query column.

    Returns:
        pd.Series: The coin information corresponding to the search_query.

    """
    return df_coins.loc[row.search_query]


def is_comment_valid_row(row: pd.Series) -> bool:
    """
    Check if a comment is valid based on the coin's start and end dates.

    Args:
        row (pd.Series): A row from the dataframe containing created and edited columns.

    Returns:
        bool: True if the comment is valid, False otherwise.

    """
    coin = get_coin_info_row(row)

    comment_date = row.edited if pd.notna(row.edited) else row.created
    start_date_valid = comment_date >= coin["start_date"]
    end_date_valid = comment_date <= coin["end_date"] if pd.notna(coin["end_date"]) else True

    return start_date_valid and end_date_valid

In [ ]:
df_reddit["test"] = df_reddit.swifter.apply(get_coin_info_row, axis=1)["test"]
df_reddit["valid"] = df_reddit.swifter.apply(is_comment_valid_row, axis=1)

df_reddit = df_reddit[df_reddit["valid"]]
df_reddit = df_reddit.drop(columns=["valid"])
df_reddit = df_reddit.reset_index(drop=True)

df_reddit.head(1)

In [ ]:
df_reddit["search_query"].value_counts()

In [ ]:
train_df = df_reddit[~df_reddit["test"]]
test_df = df_reddit[df_reddit["test"]]

In [ ]:
unique_train_coins = train_df["search_query"].unique().tolist()

In [ ]:
# Seed for reproducibility
random.seed(42)

# Create subsets
list_subsets = []
for coin in unique_train_coins:
    # Get metadata for the coin
    metadata_coin = get_coin_info(coin).copy()
    metadata_coin["start_date"] = get_first_comment_date(coin)
    metadata_coin["end_date"] = get_last_comment_date(coin)

    # Create N_OF_SPLITS_PER_SUBSET splits
    for _ in range(N_OF_SPLITS_PER_SUBSET):
        successful_split = False
        while not successful_split:
            # Get a random date between the start and end date
            random_day_diff = random.randint(0, (metadata_coin["end_date"] - metadata_coin["start_date"]).days)
            random_date = metadata_coin["start_date"].date() + pd.Timedelta(days=random_day_diff)

            # Check if there are enough comments for the subset to be valid
            if get_number_of_comments_per_coin(coin, random_date) > MIN_COMMENTS_PER_SUBSET:
                successful_split = True

        # Append the subset to the list
        list_subsets.append({"coin": coin, "split_date": random_date, "fraud": metadata_coin["fraud"]})

In [ ]:
# Drop duplicates
df_subsets = pd.DataFrame(list_subsets)
df_subsets = df_subsets.drop_duplicates()
df_subsets

In [ ]:
df_subsets.coin.value_counts()

In [ ]:
vectorizer = CountVectorizer(
    stop_words="english",
    analyzer="word",
    lowercase=True,
)

vectorizer.fit(train_df["body"])

In [ ]:
print("Number of features:", len(vectorizer.vocabulary_))

In [ ]:
def get_text_from_split(data: pd.DataFrame, split_date: str, coin: str) -> str:
    """
    Get the text data for a given coin and split date.

    Args:
        data (pd.DataFrame): The dataframe containing the data.
        split_date (str): The split date.
        coin (str): The coin name.

    Returns:
        str: The text data for the given coin and split date

    """
    split_date = pd.to_datetime(split_date, format="%Y-%m-%d")

    data = data.copy()
    data["edited"] = data["edited"].fillna(data["created"])
    data = data[data["search_query"] == coin]
    data = data[data["edited"] <= split_date]
    return data["body"].str.cat(sep=" ")

In [ ]:
# LOOCV based on each coin
metrics = defaultdict(dict)

for i, coin in tqdm(enumerate(df_subsets.coin.unique()), total=len(df_subsets.coin.unique())):
    # Get the fitting and validation splits
    fit_df = df_subsets[df_subsets["coin"] != coin]
    val_df = df_subsets[df_subsets["coin"] == coin]

    # Get the fitting data
    list_fitting_data = []
    for _, row in fit_df.iterrows():
        list_fitting_data.append(
            {
                "coin": row["coin"],
                "split_date": row["split_date"],
                "text": get_text_from_split(train_df, row["split_date"], row["coin"]),
                "fraud": row["fraud"],
            },
        )

    # Get the validation data
    list_validation_data = []
    for _, row in val_df.iterrows():
        list_validation_data.append(
            {
                "coin": row["coin"],
                "split_date": row["split_date"],
                "text": get_text_from_split(train_df, row["split_date"], row["coin"]),
                "fraud": row["fraud"],
            },
        )

    # Create the fitting and validation dataframes
    df_fitting = pd.DataFrame(list_fitting_data)
    df_validation = pd.DataFrame(list_validation_data)
    del list_fitting_data
    del list_validation_data

    # fitting Multinomial Naive Bayes
    clf = MultinomialNB()
    X_fitting = vectorizer.transform(df_fitting["text"])
    y_fitting = df_fitting["fraud"]
    clf.fit(X_fitting, y_fitting)

    # Predicting the validation set
    X_validation = vectorizer.transform(df_validation["text"])
    y_validation = df_validation["fraud"].to_numpy()
    y_pred = clf.predict_proba(X_validation)

    # calculating metrics for left out coin
    metrics[i] = {
        "coin_left_out": coin,
        "coin_split_date": df_validation["split_date"].to_list(),
        "coin_fraud": df_validation["fraud"].to_list(),
        "accuracy": 1 * (y_pred[:, 1].round() == y_validation),
        "y_pred": y_pred[:, 1].round(3),
        "y_true": y_validation,
    }

    # deleting dataframes to free up memory
    del df_fitting
    del df_validation

In [ ]:
# idea: for each coin, visualize until which date the model is able to predict frauds
import altair as alt

for i in metrics.keys():

    df_metrics = pd.DataFrame(metrics[i])
    df_metrics["coin_fraud"] = df_metrics["coin_fraud"].astype(int)
    df_metrics["coin_split_date"] = pd.to_datetime(df_metrics["coin_split_date"])

    chart = (
        alt.Chart(df_metrics)
        .mark_area(
            interpolate="step-after",
            opacity=0.6,
            line=True,
        )
        .encode(
            x=alt.X("coin_split_date", title="Date").axis(format="%Y-%m-%d", labelAngle=-90),
            y=alt.Y("accuracy", title="Accuracy", scale=alt.Scale(domain=[0,1])).axis(format="%"),
            tooltip=[
                alt.Tooltip("coin_split_date", title="Date", format="%Y-%m-%d"),
                alt.Tooltip("accuracy", title="Accuracy"),
            ],
        )
        .properties(title=f"Accuracy for {df_metrics['coin_left_out'].iloc[0]}, correct label: {"fraud" if df_metrics['coin_fraud'].iloc[0] else "not fraud"}",
                    width=500,
                    height=50)
    )

    display(chart)